In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib editdistance sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, LlamaTokenizer, LlamaForCausalLM, pipeline
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

from utils import similar_tag

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll
CUDA SETUP: CUDA runtime path found: C:\Users\panta\anaconda3\envs\nlp\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\Users\panta\anaconda3\envs\nlp did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/usr/bin'), WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/mingw-w64/bin')}
  warn(msg)


In [2]:
device = "cuda"
peft_model_id = r"ooferdoodles/text2tags-opt-1.3b"
base_model = "facebook/opt-1.3b"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16)
model.config

OPTConfig {
  "_name_or_path": "facebook/opt-1.3b",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 8192,
  "hidden_size": 2048,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 32,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "prefix": "</s>",
  "quantization_config": {
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_8bit": true
  },
  "torch_dtype": "float16",
  "transformers_version": "4.29.0.dev0",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 2048
}

In [3]:
data = load_dataset("json", data_files=r"dataset/test_data.json")
tag_dict = similar_tag.load_dict()
data

Found cached dataset json (C:/Users/panta/.cache/huggingface/datasets/json/default-5cff6cb24ab66f93/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string'],
        num_rows: 150
    })
})

In [4]:
generation_config = GenerationConfig(
    temperature=1,
    top_p=1,
    top_k=40,
    num_beams=4,
    typical_p=1,
    do_sample=True,
    max_new_tokens=300,
    use_cache=True,
    no_repeat_ngram_size=3,
    # pad_token_id=model.config.eos_token_id
    # truncation_length=2048,
    # min_length=0,
    # add_bos_token=True,
    # ban_eos_token=False,
    # skip_special_tokens=True,
    # stopping_strings=[],
    # penalty_alpha=0,
    # repetition_penalty=2.5,
    # encoder_repetition_penalty=1,
)

In [5]:
def pipe(data_point):
    # max_new_tokens = int(len(data_point['caption_string']) * max_token_scale)
    prompt = f"### Caption: {data_point['caption_string']}\n### Tags: "
    tokenized_prompt = tokenizer(prompt, return_tensors='pt').to(device)
    
    output_tokens = model.generate(
        input_ids=tokenized_prompt['input_ids'],
        generation_config=generation_config,
    )
    preds = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    pred_list = [x.strip() for x in preds.split('### Tags:')[-1].split(",")]
    corrected_tags = similar_tag.correct_tags(pred_list, tag_dict)
    data_point['tags'] = data_point['tag_string'].split(', ')
    data_point['pred_tags'] = corrected_tags
    return data_point

In [6]:
processed_data = data.map(pipe)
processed_data

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string', 'tags', 'pred_tags'],
        num_rows: 150
    })
})

In [7]:
def evaluate_accuracy(data_point):
    correct_count = len(set(data_point['tags']).intersection(data_point['pred_tags']))
    # incorrect_count = len(data_point['tags']) - correct_count
    data_point['accuracy'] = correct_count / len(data_point['tags']) * 100
    return data_point

In [8]:
evaluated_data = processed_data.map(evaluate_accuracy)
evaluated_data

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string', 'tags', 'pred_tags', 'accuracy'],
        num_rows: 150
    })
})

In [9]:
df = evaluated_data['train'].to_pandas()
df

,caption_string,tag_string,tags,pred_tags,accuracy
0,"With her mesmerizing gaze and ethereal wings, ...","1girl, breasts, closed_mouth, gloves, horns, l...","[1girl, breasts, closed_mouth, gloves, horns, ...","[bangs, blush, bra_visible_through_clothes, br...",47.058824
1,Kisaragi Ai's art features Garma Zabi from Gun...,"1boy, adjusting_hair, bow, bowtie, brown_eyes,...","[1boy, adjusting_hair, bow, bowtie, brown_eyes...","[1boy, bow, bowtie, brown_eyes, dress, hair_bo...",38.888889
2,Rojer18's art depicts Oozora Hiro from Danball...,"1boy, ahoge, belt, blue_hair, closed_mouth, cr...","[1boy, ahoge, belt, blue_hair, closed_mouth, c...","[arashio_(kancolle), belt, blue_eyes, blue_hai...",30.000000
3,"Amidst the chaos of Danganronpa, Criis-chan ca...","2girls, :d, ahoge, arm_up, bangs, black_shirt,...","[2girls, :d, ahoge, arm_up, bangs, black_shirt...","[!, arms_between_legs, bangs, black_hair, blue...",7.142857
4,The artwork is a solo depiction of the charact...,"1boy, ^_^, ahoge, blush, closed_eyes, fate/gra...","[1boy, ^_^, ahoge, blush, closed_eyes, fate/gr...","[blush, braided_ponytail, closed_eyes, fate/gr...",47.619048
...,...,...,...,...,...
145,The artwork features the character Bokuto Kout...,"1boy, artist_name, bangs, black_shirt, collarb...","[1boy, artist_name, bangs, black_shirt, collar...","[aikatsu!_(series), black_shirt, blue_eyes, el...",34.782609
146,"""Too_mizuguchi's artwork portrays Oswald Chest...","1boy, bags_under_eyes, batman_(series), black_...","[1boy, bags_under_eyes, batman_(series), black...","[!, bags_under_eyes, batman_(series), black_ha...",41.176471
147,Denjinq's artistic prowess comes to life with ...,"1boy, 5girls, aqua_hair, ass, bangs, black_hai...","[1boy, 5girls, aqua_hair, ass, bangs, black_ha...","[!, 5girls, aqua_hair, blue_shorts, blush, eye...",29.629630
148,The artwork features a single female character...,"1girl, :o, bangs, black_shirt, boots, bronya_z...","[1girl, :o, bangs, black_shirt, boots, bronya_...","[:d, elbow_gloves, gloves, grey_eyes, hair_rib...",30.303030


In [10]:
df['accuracy'].mean()

33.007494376810655

In [ ]:
df['pred_tags'][145]